In [1]:
import tensorflow as tf
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, Input, InputLayer, Dropout, BatchNormalization, Convolution2D, MaxPooling2D, GlobalMaxPool2D
from keras import activations, models, optimizers, losses
from keras.activations import relu
from keras.models import Sequential
import pandas as pd
import os

2023-12-02 11:45:45.138644: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 11:45:47.622780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 11:45:47.622967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 11:45:47.956305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 11:45:48.871350: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 11:45:48.874203: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
metadata = pd.read_csv('../UrbanSound8K/metadata/UrbanSound8K.csv')
metadata
# (x_train, y_train) = ((Xaudios, 4000), (Xaudios,))
# (x_test, y_test) = ((Yaudios, 4000), (Yaudios,))

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [3]:
labels = metadata['class'].unique()    # obtaining the class labels
print(labels)

['dog_bark' 'children_playing' 'car_horn' 'air_conditioner' 'street_music'
 'gun_shot' 'siren' 'engine_idling' 'jackhammer' 'drilling']


## CNN pre-processing

In [4]:
def one_hot_encode(label_id):
    onehot = [0]*10   # length of labels array
    onehot[label_id-1]=1
    return onehot

def reflective_padding(signal, target_duration, target_rate):
    target_duration = target_duration*target_rate
    current_duration = len(signal)
    
    # Calculate the required padding on each side
    padding_needed = target_duration - current_duration
    left_padding = padding_needed // 2
    right_padding = padding_needed - left_padding
    
    # Reflective padding on both sides
    padded_signal = np.pad(signal, (left_padding, right_padding), 'reflect')
    
    return padded_signal

In [5]:
# signal_time = 4    # each signal will have 4 seconds of duration
target_rate = 1000    # resampling frequence

set = []    # [audio, label, fold]
for index, row in metadata.iterrows():
    print(index, row)
    #for fold
    fold = row["fold"]

    # for audio
    signal, rate = librosa.load(f"../UrbanSound8K/audio/fold{fold}/"+row["slice_file_name"], sr=None)
    new_signal = librosa.resample(signal, orig_sr=rate, target_sr=target_rate)
    if len(new_signal) < 4*target_rate:
        new_signal = reflective_padding(new_signal, 4, target_rate)
    audio = new_signal[:4000]

    #for label
    label = one_hot_encode(row["classID"])

    set.append([audio, label, fold])
    print(f"Index {index} done")

set_df = pd.DataFrame(set, columns=["audio","label","fold"])
set_df.head()

0 slice_file_name    100032-3-0-0.wav
fsID                         100032
start                           0.0
end                        0.317551
salience                          1
fold                              5
classID                           3
class                      dog_bark
Name: 0, dtype: object
Index 0 done
1 slice_file_name    100263-2-0-117.wav
fsID                           100263
start                            58.5
end                              62.5
salience                            1
fold                                5
classID                             2
class                children_playing
Name: 1, dtype: object
Index 1 done
2 slice_file_name    100263-2-0-121.wav
fsID                           100263
start                            60.5
end                              64.5
salience                            1
fold                                5
classID                             2
class                children_playing
Name: 2, dtype: object
Ind

,audio,label,fold
0,"[-0.005111122, 0.00027401414, 0.0015376861, 7....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",5
1,"[0.0010924108, 0.0020332793, 0.0022091642, 0.0...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",5
2,"[-0.0011693949, 0.00056259864, -0.0002016531, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",5
3,"[-0.0010711739, -0.004255988, -0.0021388063, -...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",5
4,"[0.0018479165, 0.0054835337, 0.0029024398, -0....","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",5


In [14]:
set_df.tail()

,audio,label,fold
8727,"[-0.0019209236, -0.00206137, -0.00032925332, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7
8728,"[-0.001814239, 9.4179995e-06, 0.0049034064, -0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7
8729,"[0.011096267, -0.006043774, -0.009053946, -0.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7
8730,"[-0.00019298878, 0.002788129, -0.0014333483, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7
8731,"[0.0008923189, 0.0017864043, 0.0011411505, 0.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7


# MLP

In [32]:
mlp = Sequential()
mlp.add(Dense(200, activation='relu',input_shape=(4000, ))) # input layer  #4000 = sample rate 1000 * 4sec audio
mlp.add(Dropout(0.5))
mlp.add(BatchNormalization())
mlp.add(Dense(200,activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(BatchNormalization())
mlp.add(Dense(200,activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(BatchNormalization())
mlp.add(Dense(100,activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax')) # output layer  #10 = n_class

mlp.compile(loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer='adam')
            

# summary
mlp.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 200)               800200    
                                                                 
 dropout_17 (Dropout)        (None, 200)               0         
                                                                 
 batch_normalization_21 (Ba  (None, 200)               800       
 tchNormalization)                                               
                                                                 
 dense_39 (Dense)            (None, 200)               40200     
                                                                 
 dropout_18 (Dropout)        (None, 200)               0         
                                                                 
 batch_normalization_22 (Ba  (None, 200)               800       
 tchNormalization)                                    

# CNN

In [23]:
nclass = 10
inp = Input(shape=(157, 320, 1))
norm_inp = BatchNormalization()(inp)
audio = Convolution2D(16, kernel_size=(3, 7), activation=activations.relu)(norm_inp)
audio = Convolution2D(16, kernel_size=(3, 7), activation=activations.relu)(audio)
audio = MaxPooling2D(pool_size=(3, 7))(audio)
audio = Dropout(rate=0.1)(audio)
audio = Convolution2D(32, kernel_size=3, activation=activations.relu)(audio)
audio = Convolution2D(32, kernel_size=3, activation=activations.relu)(audio)
audio = MaxPooling2D(pool_size=(3, 3))(audio)
audio = Dropout(rate=0.1)(audio)
audio = Convolution2D(128, kernel_size=3, activation=activations.relu)(audio)
audio = GlobalMaxPool2D()(audio)
audio = Dropout(rate=0.1)(audio)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(audio))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

cnn = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

cnn.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 157, 320, 1)]     0         
                                                                 
 batch_normalization (Batch  (None, 157, 320, 1)       4         
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 155, 314, 16)      352       
                                                                 
 conv2d_1 (Conv2D)           (None, 153, 308, 16)      5392      
                                                                 
 max_pooling2d (MaxPooling2  (None, 51, 44, 16)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 51, 44, 16)        0     

## main run

In [33]:
for i in range(10):
    test_df = set_df.drop(set_df[set_df['fold'] != i+1].index)
    train_df = set_df.drop(set_df[set_df['fold'] == i+1].index)

    X_train = np.array(train_df['audio'].tolist())
    X_test = np.array(test_df['audio'].tolist())
    y_train = np.array(train_df['label'].tolist())
    y_test = np.array(test_df['label'].tolist())

    n_epochs = 100
    n_batch_size = 32
    mlp.fit(X_train, y_train, batch_size=n_batch_size, epochs=n_epochs, validation_data=(X_test, y_test))
        
    test_accuracy=mlp.evaluate(X_test,y_test,verbose=0)
    print(f"Accuracy using fold {i+1} as validation: {test_accuracy[1]}")
    

Epoch 1/100


2023-12-02 13:02:32.256645: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 125744000 exceeds 10% of free system memory.


246/246 [==============================] - 5s 12ms/step - loss: 2.6580 - accuracy: 0.1355 - val_loss: 2.3262 - val_accuracy: 0.1168
Epoch 2/100
246/246 [==============================] - 3s 11ms/step - loss: 2.3290 - accuracy: 0.1545 - val_loss: 2.2777 - val_accuracy: 0.1409
Epoch 3/100
246/246 [==============================] - 3s 12ms/step - loss: 2.2379 - accuracy: 0.1627 - val_loss: 2.2764 - val_accuracy: 0.1386
Epoch 4/100
246/246 [==============================] - 3s 11ms/step - loss: 2.1970 - accuracy: 0.1686 - val_loss: 2.2725 - val_accuracy: 0.1535
Epoch 5/100
246/246 [==============================] - 3s 11ms/step - loss: 2.1740 - accuracy: 0.1779 - val_loss: 2.2692 - val_accuracy: 0.1501
Epoch 6/100
246/246 [==============================] - 3s 11ms/step - loss: 2.1530 - accuracy: 0.1811 - val_loss: 2.2909 - val_accuracy: 0.1546
Epoch 7/100
246/246 [==============================] - 3s 11ms/step - loss: 2.1468 - accuracy: 0.1883 - val_loss: 2.2710 - val_accuracy: 0.1638
Epoc

2023-12-02 13:07:17.168180: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 125504000 exceeds 10% of free system memory.


246/246 [==============================] - 3s 12ms/step - loss: 1.1338 - accuracy: 0.6406 - val_loss: 0.9539 - val_accuracy: 0.7511
Epoch 2/100
246/246 [==============================] - 3s 11ms/step - loss: 1.0733 - accuracy: 0.6493 - val_loss: 0.9282 - val_accuracy: 0.7286
Epoch 3/100
246/246 [==============================] - 3s 11ms/step - loss: 1.0403 - accuracy: 0.6571 - val_loss: 0.9471 - val_accuracy: 0.7151
Epoch 4/100
246/246 [==============================] - 3s 11ms/step - loss: 1.0096 - accuracy: 0.6655 - val_loss: 0.9822 - val_accuracy: 0.6667
Epoch 5/100
246/246 [==============================] - 3s 11ms/step - loss: 0.9874 - accuracy: 0.6684 - val_loss: 0.9181 - val_accuracy: 0.7106
Epoch 6/100
246/246 [==============================] - 3s 11ms/step - loss: 0.9876 - accuracy: 0.6618 - val_loss: 0.9669 - val_accuracy: 0.7128
Epoch 7/100
246/246 [==============================] - 3s 11ms/step - loss: 0.9696 - accuracy: 0.6694 - val_loss: 0.9718 - val_accuracy: 0.7128
Epoc

2023-12-02 13:11:58.150483: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 124912000 exceeds 10% of free system memory.


244/244 [==============================] - 3s 12ms/step - loss: 0.8452 - accuracy: 0.7219 - val_loss: 0.9136 - val_accuracy: 0.7730
Epoch 2/100
244/244 [==============================] - 3s 12ms/step - loss: 0.8013 - accuracy: 0.7306 - val_loss: 0.7474 - val_accuracy: 0.7989
Epoch 3/100
244/244 [==============================] - 3s 11ms/step - loss: 0.7702 - accuracy: 0.7372 - val_loss: 0.8137 - val_accuracy: 0.7643
Epoch 4/100
244/244 [==============================] - 3s 11ms/step - loss: 0.7453 - accuracy: 0.7468 - val_loss: 0.7127 - val_accuracy: 0.8465
Epoch 5/100
244/244 [==============================] - 3s 11ms/step - loss: 0.7412 - accuracy: 0.7487 - val_loss: 0.8141 - val_accuracy: 0.8432
Epoch 6/100
244/244 [==============================] - 3s 11ms/step - loss: 0.7323 - accuracy: 0.7537 - val_loss: 0.7017 - val_accuracy: 0.8032
Epoch 7/100
244/244 [==============================] - 3s 11ms/step - loss: 0.7179 - accuracy: 0.7523 - val_loss: 0.7415 - val_accuracy: 0.7795
Epoc

2023-12-02 13:16:36.539183: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123872000 exceeds 10% of free system memory.


242/242 [==============================] - 3s 12ms/step - loss: 0.6573 - accuracy: 0.7786 - val_loss: 0.7578 - val_accuracy: 0.7697
Epoch 2/100
242/242 [==============================] - 3s 12ms/step - loss: 0.6491 - accuracy: 0.7785 - val_loss: 0.6834 - val_accuracy: 0.7828
Epoch 3/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6567 - accuracy: 0.7707 - val_loss: 0.6876 - val_accuracy: 0.8101
Epoch 4/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6281 - accuracy: 0.7861 - val_loss: 0.6682 - val_accuracy: 0.7707
Epoch 5/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6393 - accuracy: 0.7805 - val_loss: 0.7058 - val_accuracy: 0.7687
Epoch 6/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6220 - accuracy: 0.7858 - val_loss: 0.7286 - val_accuracy: 0.7414
Epoch 7/100
242/242 [==============================] - 3s 11ms/step - loss: 0.6067 - accuracy: 0.7979 - val_loss: 0.7080 - val_accuracy: 0.7475
Epoc